# Spark
## HDFS, Impala, Jupyter

## Vstup

- Pomocí Spark.sql poslat select na libovolnou LinFada tabulku a vytvořit tempView.
- CSV file (třeba Titanic)

## Výstup

- Jupyter notebook na GIT/email.

In [135]:
!kdestroy
!kinit -kt ../Simona.Uhlikova.keytab Simona.Uhlikova
!klist

Ticket cache: FILE:/tmp/krb5cc_1512860934
Default principal: Simona.Uhlikova@DATALAB.ADASTRA.CZ

Valid starting       Expires              Service principal
04/21/2023 07:18:42  04/22/2023 07:18:42  krbtgt/DATALAB.ADASTRA.CZ@DATALAB.ADASTRA.CZ


In [136]:
# import modules
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
spark = SparkSession.builder.appName("app").getOrCreate()

### 1) Data z tempView uložit do df
### 2) Vypsat schema df
### 3) Zobrazit df (vyzkoušet truncate), vypsat počet řádků 

In [138]:
def linfada_data(database_name, table_name) -> DataFrame:

    # Load the data from the tempView into a DataFrame
    df = spark.sql(f"SELECT * FROM {database_name}.{table_name}")
    
    df.createOrReplaceTempView(table_name)

    # print the schema
    df.printSchema()

    # show contents of the DataFrame with the number of rows
    df.show(truncate=True)
    print(f"Počet riadkov: {df.count()}")
    
    return df

In [139]:
#calling functions
linfada = linfada_data("linfada_mart","lin_downloaded_posts")

23/04/21 07:20:55 WARN conf.HiveConf: HiveConf of name hive.masking.algo does not exist
Hive Session ID = 4d3c0539-dbbb-4e31-ac8c-1b5aa2a386d4


root
 |-- id_downloaded_post: long (nullable = true)
 |-- post_text: string (nullable = true)
 |-- post_hashtags: string (nullable = true)
 |-- unique_impressions_count: integer (nullable = true)
 |-- share_count: integer (nullable = true)
 |-- engagement: float (nullable = true)
 |-- click_count: integer (nullable = true)
 |-- like_count: integer (nullable = true)
 |-- impression_count: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- last_edit_time: timestamp (nullable = true)
 |-- s_download_time: timestamp (nullable = true)
 |-- s_date: integer (nullable = true)



23/04/21 07:20:56 WARN security.ShellBasedUnixGroupsMapping: Some group names for 'simona.uhlikova' are not resolvable. id: cannot find name for group ID 1512800513
id: cannot find name for group ID 1512800513

[Stage 0:>                                                          (0 + 0) / 1]23/04/21 07:21:12 WARN cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/21 07:21:27 WARN cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/21 07:21:42 WARN cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/04/21 07:21:57 WARN cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient

+-------------------+--------------------+--------------------+------------------------+-----------+----------+-----------+----------+----------------+-------------+-------------------+-------------------+-------------------+------+
| id_downloaded_post|           post_text|       post_hashtags|unique_impressions_count|share_count|engagement|click_count|like_count|impression_count|comment_count|       publish_time|     last_edit_time|    s_download_time|s_date|
+-------------------+--------------------+--------------------+------------------------+-----------+----------+-----------+----------+----------------+-------------+-------------------+-------------------+-------------------+------+
|6879515477618565120|Baví tě správa ap...|sql,linux,windows...|                       3|          0|       0.0|          0|         2|               3|            0|2021-12-22 20:19:18|2021-12-22 20:49:56|2023-02-22 08:27:51|202302|
|6879515477618565120|Baví tě správa ap...|sql,linux,windows...|     

[Stage 1:=============================>                             (1 + 1) / 2]

Počet riadkov: 20688


### 4) Načíst přiložený CSV file s předem nadefinovaným schématem pomocí StructType (s hlavičkou, delimiterem)

In [140]:
def read_titanic_csv_with_struct(titanic_path) -> DataFrame:
    # Load Titanic CSV file with pre-defined StructType
    titanic_schema = StructType([
        StructField("PassengerId", IntegerType(), True),
        StructField("Survived", IntegerType(), True),
        StructField("Pclass", IntegerType(), True),
        StructField("Name", StringType(), True),
        StructField("Sex", StringType(), True),
        StructField("Age", FloatType(), True),
        StructField("SibSp", IntegerType(), True),
        StructField("Parch", IntegerType(), True),
        StructField("Ticket", StringType(), True),
        StructField("Fare", FloatType(), True),
        StructField("Cabin", StringType(), True),
        StructField("Embarked", StringType(), True)
    ])

    # načtení dat s definovaným schématem
    titanic_struct_df = spark.read.format("csv") \
        .option("header", True) \
        .option("delimiter", ",") \
        .schema(titanic_schema) \
        .load(titanic_path)
    
    # Show results
    titanic_struct_df.show(truncate=True)

    # Print schema
    print("StructType schema:")
    titanic_struct_df.printSchema()

    # Print row count
    print("Row count:")
    print(f"StructType: {titanic_struct_df.count()}")

    return titanic_struct_df

In [141]:
titanic_struct_df = read_titanic_csv_with_struct("Data/titanic.csv")

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

### 5) Načíst přiložený CSV file s předem nadefinovaným schématem pomocí DDL stringu

In [142]:
def read_titanic_csv_with_ddl(titanic_path) -> DataFrame:

    # Define schema using DDL string
    ddl_schema = """
        PassengerId INT,
        Survived INT,
        Pclass INT,
        Name STRING,
        Sex STRING,
        Age DOUBLE,
        SibSp INT,
        Parch INT,
        Ticket STRING,
        Fare DOUBLE,
        Cabin STRING,
        Embarked STRING
    """

    # Load CSV file with defined schema
    titanic_ddl_df = spark.read.format("csv") \
        .option("header", "true") \
        .option("delimiter", ",") \
        .schema(ddl_schema) \
        .load(titanic_path)
    
    # Show results
    titanic_ddl_df.show(truncate=True)
    
    # Print schema
    print("DDL schema:")
    titanic_ddl_df.printSchema()
    
    # Print row count
    print("Row count:")
    print(f"DDL: {titanic_ddl_df.count()}")
    
    return titanic_ddl_df

In [143]:
titanic_ddl_df = read_titanic_csv_with_ddl("Data/titanic.csv")

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

### 6) Načíst přiložený CSV file za pomoci inferSchema (porovnat s ostatníma)

- Funkcia načíta CSV súbor a automaticky určí typy dát pre každý stĺpec

In [144]:
def read_titanic_csv_with_infer_schema(titanic_path) -> DataFrame:
    spark = SparkSession.builder.appName("TitanicApp").getOrCreate()

    # Load CSV file with inferred schema
    titanic_infer_df = spark.read.format("csv") \
        .option("header", "true") \
        .option("delimiter", ",") \
        .option("inferSchema", "true") \
        .load(titanic_path)
    
    # Show results
    titanic_infer_df.show(truncate=True)

    # Print schema
    print("Inferred schema:")
    titanic_infer_df.printSchema()

    # Print row count
    print("Row count:")
    print(f"Inferred schema: {titanic_infer_df.count()}")

    return titanic_infer_df

In [145]:
titanic_infer_df = read_titanic_csv_with_infer_schema("Data/titanic.csv")

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

### 7) Využití selectu na více sloupců se vstupem jako string, pyspark.sql.functions.col, colRegex

- colRegex function was added in spark version 3.1.0. 

In [146]:
def titanic_df_select(titanic_df, select_string)-> DataFrame:
    # Select columns using string
    selected_cols = [col(c) for c in select_string.split(",")]
    select_df = titanic_df.select(selected_cols)

    # Select columns using col function
    col_df = titanic_df.select(col("Name"), col("Age"), col("Fare"))

    # Select columns using colRegex function
    #regex_df = titanic_df.select(colRegex("Na.e"), colRegex("^Age$"), colRegex("F.r$"))

    return select_df, col_df # ,regex_df

In [147]:
# Call the function
select_df, col_df = titanic_df_select(titanic_struct_df, "Name,Age,Fare")

# Show the results
select_df.show()
col_df.show()
#regex_df.show()

+--------------------+----+-------+
|                Name| Age|   Fare|
+--------------------+----+-------+
|Braund, Mr. Owen ...|22.0|   7.25|
|Cumings, Mrs. Joh...|38.0|71.2833|
|Heikkinen, Miss. ...|26.0|  7.925|
|Futrelle, Mrs. Ja...|35.0|   53.1|
|Allen, Mr. Willia...|35.0|   8.05|
|    Moran, Mr. James|null| 8.4583|
|McCarthy, Mr. Tim...|54.0|51.8625|
|Palsson, Master. ...| 2.0| 21.075|
|Johnson, Mrs. Osc...|27.0|11.1333|
|Nasser, Mrs. Nich...|14.0|30.0708|
|Sandstrom, Miss. ...| 4.0|   16.7|
|Bonnell, Miss. El...|58.0|  26.55|
|Saundercock, Mr. ...|20.0|   8.05|
|Andersson, Mr. An...|39.0| 31.275|
|Vestrom, Miss. Hu...|14.0| 7.8542|
|Hewlett, Mrs. (Ma...|55.0|   16.0|
|Rice, Master. Eugene| 2.0| 29.125|
|Williams, Mr. Cha...|null|   13.0|
|Vander Planke, Mr...|31.0|   18.0|
|Masselmani, Mrs. ...|null|  7.225|
+--------------------+----+-------+
only showing top 20 rows

+--------------------+----+-------+
|                Name| Age|   Fare|
+--------------------+----+-------+
|B

### 8) Vyselectit všechny sloupce, pouze prvních 5 indexů, selectExpr

In [148]:
def select_first_five_columns(titanic_df):
    return titanic_df.selectExpr(titanic_df.columns[:5])

In [149]:
select_first_five_columns(titanic_struct_df).show(5)

+-----------+--------+------+--------------------+------+
|PassengerId|Survived|Pclass|                Name|   Sex|
+-----------+--------+------+--------------------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|
|          2|       1|     1|Cumings, Mrs. Joh...|female|
|          3|       1|     3|Heikkinen, Miss. ...|female|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|
|          5|       0|     3|Allen, Mr. Willia...|  male|
+-----------+--------+------+--------------------+------+
only showing top 5 rows



### 9) Využití funkce filter, negace podmínky, && + ||, isin, contains, endswitch, like, array_contains 

In [150]:
# split "Cabin" column into array
titanic_struct_df = titanic_struct_df.withColumn("CabinArray", split(col("Cabin"), ","))

# drop original "Cabin" column
titanic_struct_df = titanic_struct_df.drop("Cabin")

# show result
titanic_struct_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|CabinArray|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+----------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|       S|      null|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|       C|     [C85]|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|       S|      null|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1|       S|    [C123]|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05|       S|      null|
|          6|       0|     3|    Moran, 

In [151]:
def titanic_dataframe_filter(titanic_df) -> DataFrame:
    # Filter based on Age and Sex columns using '&&', '||', and negation
    filtered_df = titanic_df.filter((col("Age") > 20) & ((col("Sex") == "female") | ~(col("Age") > 50)))

    # Filter based on Embarked column using isin function
    filtered_df = filtered_df.filter(col("Embarked").isin(["C", "Q"]))

    # Filter based on Name column using contains and endswith functions
    filtered_df = filtered_df.filter(col("Name").contains("Mrs.") | col("Name").endswith("ovich"))

    # Filter based on Cabin column using like function
    filtered_df = filtered_df.filter(col("Name").like("%M%"))

    # Filter based on Ticket column using array_contains function
    filtered_df = titanic_df.filter(array_contains(col("CabinArray"), "B18"))

    return filtered_df

In [152]:
filtered_df = titanic_dataframe_filter(titanic_struct_df).show()

+-----------+--------+------+--------------------+------+----+-----+-----+------+-------+--------+----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|   Fare|Embarked|CabinArray|
+-----------+--------+------+--------------------+------+----+-----+-----+------+-------+--------+----------+
|        330|       1|     1|Hippach, Miss. Je...|female|16.0|    0|    1|111361|57.9792|       C|     [B18]|
|        524|       1|     1|Hippach, Mrs. Lou...|female|44.0|    0|    1|111361|57.9792|       C|     [B18]|
+-----------+--------+------+--------------------+------+----+-----+-----+------+-------+--------+----------+



### 10) Funkce where

- V tejto funkcii sú riadky filtrované pomocou funkcie where namiesto funkcie filter.

In [153]:
def filter_titanic_data(titanic_df) -> DataFrame:
    filtered_df = titanic_df.where(col("Age") > 20) \
        .where((col("Sex") == "female") | ~(col("Age") > 50)) \
        .where(col("Embarked").isin(["C", "Q"])) \
        .where(col("Name").contains("Mrs.") | col("Name").endswith("ovich")) \
        .where(array_contains(col("CabinArray"), 'B18'))
        
    return filtered_df

In [154]:
filtered_df = titanic_dataframe_filter(titanic_struct_df).show()

+-----------+--------+------+--------------------+------+----+-----+-----+------+-------+--------+----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|   Fare|Embarked|CabinArray|
+-----------+--------+------+--------------------+------+----+-----+-----+------+-------+--------+----------+
|        330|       1|     1|Hippach, Miss. Je...|female|16.0|    0|    1|111361|57.9792|       C|     [B18]|
|        524|       1|     1|Hippach, Mrs. Lou...|female|44.0|    0|    1|111361|57.9792|       C|     [B18]|
+-----------+--------+------+--------------------+------+----+-----+-----+------+-------+--------+----------+



### 11) Vytvořit/přejmenovat sloupce s withColumnRenamed, withColumn
### 12) Využít Drop, dropDuplicates (subset)
### 13) Projít si agregace pomocí groupBy (agg,avg, count, sum) 
### 14) Řadit df za pomoci Sort, orderBy
### 15) Zkusit si distinct, limit

In [155]:
def manipulate_df(df) -> DataFrame:
    # 11) Vytvořit/přejmenovat sloupce s withColumnRenamed, withColumn
    df = df.withColumnRenamed("Pclass", "Class")\
           .withColumn("NewAge", col("Age") + 10)
    
    # 12) Využít Drop, dropDuplicates (subset)
    df = df.drop("SibSp", "Parch")\
           .dropDuplicates(subset=["Name", "NewAge"])
    
    # 13) Projít si agregace pomocí groupBy (agg,avg, count, sum)
    df = df.groupBy("Class", "Sex")\
           .agg(avg("NewAge"), count("Name"), sum("Fare"))
    
    # 14) Řadit df za pomoci Sort, orderBy
    df = df.sort(col("Class"))\
           .orderBy(col("sum(Fare)").desc())
    
    # 15) Zkusit si distinct, limit
    df = df.distinct()\
           .limit(6)
    
    return df

In [156]:
df_updated = manipulate_df(titanic_struct_df).show()

[Stage 25:=====================================================>(198 + 2) / 200]

+-----+------+------------------+-----------+------------------+
|Class|   Sex|       avg(NewAge)|count(Name)|         sum(Fare)|
+-----+------+------------------+-----------+------------------+
|    1|female| 44.61176470588235|         94| 9975.824983596802|
|    3|female|             31.75|        144|2321.1086139678955|
|    2|  male| 40.74070706993643|        108|2132.1125020980835|
|    2|female|38.722972972972975|         76|1669.7292032241821|
|    1|  male|51.281386139369246|        122| 8201.587522506714|
|    3|  male| 36.50758893310788|        347| 4393.586539268494|
+-----+------+------------------+-----------+------------------+



### 16) Join dvou dataframů (inner, left, outer, semi - hlavně jak fungují)

In [157]:
titanic_ddl_df
titanic_struct_df

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: float, SibSp: int, Parch: int, Ticket: string, Fare: float, Embarked: string, CabinArray: array<string>]

- Inner Join: Výsledkem inner joinu jsou pouze řádky, které mají odpovídající hodnoty v obou spojovaných DataFramech. Tento typ joinu můžeme provést pomocí parametru "inner".

In [158]:
inner_joined_df = titanic_struct_df.join(titanic_ddl_df, titanic_struct_df.PassengerId == titanic_ddl_df.PassengerId, how = "inner")
inner_joined_df.show(5)

[Stage 29:>                                                         (0 + 1) / 1]

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+----------+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|CabinArray|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+----------+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|       S|      null|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|

In [159]:
titanic_struct_df

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: float, SibSp: int, Parch: int, Ticket: string, Fare: float, Embarked: string, CabinArray: array<string>]

In [160]:
titanic_ddl_df

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

- Left Join: Výsledkem left joinu jsou všechny řádky z levého DataFramu a odpovídající řádky z pravého DataFramu. Pokud neexistuje odpovídající řádek v pravém DataFramu, budou výsledné hodnoty null. Tento typ joinu můžeme provést pomocí parametru "left".

In [161]:
left_joined_df = titanic_struct_df.join(titanic_ddl_df, titanic_struct_df.PassengerId == titanic_ddl_df.PassengerId, how = "left")
left_joined_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+----------+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|CabinArray|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+----------+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|       S|      null|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|

- Right Join: Výsledkem right joinu jsou všechny řádky z pravého DataFramu a odpovídající řádky z levého DataFramu. Pokud neexistuje odpovídající řádek v levém DataFramu, budou výsledné hodnoty null. Tento typ joinu můžeme provést pomocí parametru "right".

In [162]:
right_joined_df = titanic_struct_df.join(titanic_ddl_df, titanic_struct_df.PassengerId == titanic_ddl_df.PassengerId, how = "right")
right_joined_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+----------+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|CabinArray|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+----------+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|       S|      null|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|

- Outer Join: Výsledkem outer joinu jsou všechny řádky z obou spojovaných DataFrame, pokud neexistuje odpovídající řádek v jednom z DataFrame, budou výsledné hodnoty null. Tento typ joinu můžeme provést pomocí parametru "outer".

In [163]:
outer_joined_df = titanic_struct_df.join(titanic_ddl_df, titanic_struct_df.PassengerId == titanic_ddl_df.PassengerId, how = "outer")
outer_joined_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------+-------+--------+----------+-----------+--------+------+--------------------+------+----+-----+-----+----------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|    Ticket|   Fare|Embarked|CabinArray|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|    Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------+-------+--------+----------+-----------+--------+------+--------------------+------+----+-----+-----+----------+-------+-----+--------+
|        148|       0|     3|"Ford, Miss. Robi...|female| 9.0|    2|    2|W./C. 6608| 34.375|       S|      null|        148|       0|     3|"Ford, Miss. Robi...|female| 9.0|    2|    2|W./C. 6608| 34.375| null|       S|
|        463|       0|     1|   Gee, Mr. Arthur H|  male|47.0|    0|    0|    111320|   38.5|       S|     [E63]|   

- Leftsemi Join: Semi join je speciální typ joinu, který vrací pouze řádky z levého DataFramu, které mají odpovídající hodnoty v pravém DataFramu. Výstup neobsahuje žádné sloupce z pravého DataFramu.

In [164]:
semi_joined_df = titanic_struct_df.join(titanic_ddl_df.select("PassengerId").distinct(), ["PassengerId"], "leftsemi")
semi_joined_df.show(5)

[Stage 41:===================================>                  (131 + 3) / 200]

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|CabinArray|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+----------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|       S|      null|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|       C|     [C85]|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|       S|      null|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1|       S|    [C123]|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05|       S|      null|
+-----------+--------+------+-----------

### 17) Cast data typů

In [165]:
titanic_ddl_df = titanic_ddl_df.withColumn("Age", col("Age").cast("integer"))

titanic_ddl_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



### 18) Collect, explode, split, lit

- collect() is an action operation that is used to retrieve all the elements of the dataset as an Array of Row type (from all nodes) to the driver node

In [166]:
def collect_rows(df) -> DataFrame:
    
    rows = df.collect()
    
    return rows

In [167]:
rows = collect_rows(titanic_struct_df)

In [168]:
print(rows)

[Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Embarked='S', CabinArray=None), Row(PassengerId=2, Survived=1, Pclass=1, Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age=38.0, SibSp=1, Parch=0, Ticket='PC 17599', Fare=71.2833023071289, Embarked='C', CabinArray=['C85']), Row(PassengerId=3, Survived=1, Pclass=3, Name='Heikkinen, Miss. Laina', Sex='female', Age=26.0, SibSp=0, Parch=0, Ticket='STON/O2. 3101282', Fare=7.925000190734863, Embarked='S', CabinArray=None), Row(PassengerId=4, Survived=1, Pclass=1, Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age=35.0, SibSp=1, Parch=0, Ticket='113803', Fare=53.099998474121094, Embarked='S', CabinArray=['C123']), Row(PassengerId=5, Survived=0, Pclass=3, Name='Allen, Mr. William Henry', Sex='male', Age=35.0, SibSp=0, Parch=0, Ticket='373450', Fare=8.050000190734863, Embarked='S', CabinArray=None), Row(P

- explode(): Returns a new row for each element in the given array or map.

In [169]:
def explode_column(df, column) -> DataFrame:

    exploded_df = df.select(explode(split(col(column), ",")).alias(column))
    
    return exploded_df

In [170]:
cabin_df = explode_column(titanic_ddl_df, "Cabin")

In [171]:
cabin_df.show(5)

+-----+
|Cabin|
+-----+
|  C85|
| C123|
|  E46|
|   G6|
| C103|
+-----+
only showing top 5 rows



- split(): split DataFrame string Column into multiple columns.

In [172]:
def split_column(df, column, delimiter) -> DataFrame:
    
    splitted_df = df.withColumn('last_name', split(df[column], delimiter).getItem(0)) \
       .withColumn('first_name', split(df[column], delimiter).getItem(1)) 
    
    return splitted_df

In [173]:
name_df = split_column(titanic_struct_df, "Name", ",")

In [174]:
name_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+----------+---------+--------------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|CabinArray|last_name|          first_name|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+----------+---------+--------------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|       S|      null|   Braund|     Mr. Owen Harris|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|       C|     [C85]|  Cumings| Mrs. John Bradle...|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|       S|      null|Heikkinen|         Miss. Laina|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   

- lit(): used to add a new column to DataFrame by assigning a literal or constant value

In [175]:
def add_constant_column(df, new_col_name, column_name, value) -> DataFrame:
  
    const_df = df.withColumn(new_col_name, when(col(column_name) == value,lit("0")).otherwise(lit("1")))
    
    return const_df

In [176]:
const_df = add_constant_column(titanic_struct_df, "New", "Sex", "male")

In [177]:
const_df.show(truncate=False)

+-----------+--------+------+-------------------------------------------------------+------+----+-----+-----+----------------+-------+--------+----------+---+
|PassengerId|Survived|Pclass|Name                                                   |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Embarked|CabinArray|New|
+-----------+--------+------+-------------------------------------------------------+------+----+-----+-----+----------------+-------+--------+----------+---+
|1          |0       |3     |Braund, Mr. Owen Harris                                |male  |22.0|1    |0    |A/5 21171       |7.25   |S       |null      |0  |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)    |female|38.0|1    |0    |PC 17599        |71.2833|C       |[C85]     |1  |
|3          |1       |3     |Heikkinen, Miss. Laina                                 |female|26.0|0    |0    |STON/O2. 3101282|7.925  |S       |null      |1  |
|4          |1       |1     |Futrelle, Mrs. Ja

### 19) Spark UDF - zkusit si aplikovat jednu funkci

In [178]:
def is_embarked_cherbourg(column_name):
    """
    UDF to indicate whether the passenger embarked in Cherbourg
    """
    if column_name == "C":
        return True
    else:
        return False

udf_is_embarked_cherbourg = udf(is_embarked_cherbourg, BooleanType())

In [179]:
titanic_with_embarked_cherbourg = titanic_struct_df.withColumn("embarked_cherbourg", udf_is_embarked_cherbourg(col("Embarked")))

In [180]:
titanic_with_embarked_cherbourg.show(5)

[Stage 47:>                                                         (0 + 1) / 1]

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+----------+------------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|CabinArray|embarked_cherbourg|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+----------+------------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|       S|      null|             false|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|       C|     [C85]|              true|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|       S|      null|             false|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1|       S|    [C123]|             false|
|          5|       0|     

### 20) Uložit jako Parquet s partition

In [182]:
titanic_struct_df.write.partitionBy("Embarked").parquet("titanic_partitioned")

In [ ]:
titanic_struct_df.write.partitionBy("Sex").parquet("titanic_partitioned")

### 21) Stáhnout Parquet file z HDFS pomocí Jupyteru

In [189]:
hdfs_path = "titanic_partitioned"

In [190]:
df = spark.read.format("parquet").load(hdfs_path)
df.show()

[Stage 50:>                                                         (0 + 1) / 1]

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|CabinArray|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+----------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|      null|       S|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|      null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1|    [C123]|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05|      null|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|           17463|51.8625|     [E46]|       S|
|          8|       0|     3|Palsson, Ma

In [191]:
spark.stop()

# Main function

In [ ]:
def main():

    #calling functions
    linfada = linfada_data("linfada_mart","lin_downloaded_posts")
    titanic_struct_df = read_titanic_csv_with_struct("Data/titanic.csv")
    titanic_ddl_df = read_titanic_csv_with_ddl("Data/titanic.csv")
    titanic_infer_df = read_titanic_csv_with_infer_schema("Data/titanic.csv")
    select_df, col_df = titanic_df_select(titanic_struct_df, "Name,Age,Fare")
    first_five_columns = select_first_five_columns(titanic_struct_df)
    filtered_df = titanic_dataframe_filter(titanic_struct_df)
    cabin_df = explode_column(titanic_ddl_df, "Cabin")
    name_df = split_column(titanic_struct_df, "Name", ",")
    const_df = add_constant_column(titanic_struct_df, "New", "Sex", "male")

    
    # end spark
    spark.stop()
    
    return None

In [ ]:
# Starting point for PySpark
if __name__ == '__main__':
    main()
    sys.exit()